# TP especial Fundamentos de la Ciencia de Datos<br>
### Grupo 7: Buralli, Todesco, Antúnez


Empezaremos descargando y leyendo los archivos mandados por la cátedra

In [8]:
covers_dataset = pd.read_csv('Covers.csv')

Hacemos una vista general para saber de que se trata el dataset, que significan cada una de sus columnas

In [10]:
covers_dataset.head()

,Track,Artist,Duration,Time_Signature,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Popularity,Year
0,Everybody Everybody,Black Box,4:11,4,0.645,0.795,7,-12.095,1,0.0527,0.3470,0.003890,0.1040,0.535,112.067,87,1990
1,Close To You,Maxi Priest,3:45,4,0.590,0.727,6,-7.955,1,0.0706,0.0259,0.000065,0.0827,0.356,121.966,85,1990
2,Do Me!,Bell Biv DeVoe,3:56,4,0.615,0.534,9,-6.719,0,0.0386,0.1060,0.000018,0.0607,0.193,135.917,84,1990
3,Everything,Jody Watley,4:05,4,0.704,0.225,6,-14.454,0,0.0994,0.9020,0.657000,0.1060,0.243,120.006,82,1990
4,What It Takes,Aerosmith,3:21,4,0.672,0.655,10,-5.021,0,0.0310,0.0362,0.000000,0.1170,0.555,134.948,82,1990


### Descripción de las variables<br>
* ```Track```: nombre de la canción
* ```Artist```: nombre del artista o intérprete
* ```Duration```: duración en minutos de la canción
* ```Time_Signature```: número de pulsaciones por compás
* ```Danceability```: medida de que tan bailable es la canción(entre 0 y 1)
* ```Energy```: medida de que tan enérgica es la canción(entre 0 y 1)
* ```Key```: tonalidad de la canción, número entero
* ```Loudness```: volumen de la canción, en decibelios
* ```Mode```: tono mayor o menor(0 o 1, respectivamente)
* ```Speechiness```: medida de presencia de palabras habladas en las canciones, valores altos indican una alta presencia de estas palabras
* ```Acoustiness```: mide que tan acústica es la pista
* ```Instrumentalness```: mide la presencia de voces en las canciones, valores más altos indican una canción con mayor parte instrumental
* ```Liveness```: probabilidad de que dicha canción se haya interpretado en vivo, niveles más altos indican mayor presencia de voces de la audiencia
* ```Valance```: medida de la positividad de la canción, niveles más altos indican presencias de mlodías alegres
* ```Tempo```: velocidad de la pista, medida en beats por minutos(BPM)
* ```Popularity```: puntuación de la canción que mide su popularidad
* ```Year```: año de lanzamiento

Ahora nos ocuparemos de comprobar que los tipos de las variables sean adecuados a lo que representan. Para ello, usamos el método info()

In [9]:
covers_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track             919 non-null    object 
 1   Artist            919 non-null    object 
 2   Duration          919 non-null    object 
 3   Time_Signature    919 non-null    int64  
 4   Danceability      919 non-null    float64
 5   Energy            919 non-null    float64
 6   Key               919 non-null    int64  
 7   Loudness          919 non-null    float64
 8   Mode              919 non-null    int64  
 9   Speechiness       919 non-null    float64
 10  Acousticness      919 non-null    float64
 11  Instrumentalness  919 non-null    float64
 12  Liveness          919 non-null    float64
 13  Valence           919 non-null    float64
 14  Tempo             919 non-null    float64
 15  Popularity        919 non-null    int64  
 16  Year              919 non-null    int64  
dt

Duración = object?? Corregir

Lo veo bien creo

In [11]:
covers_dataset.isna().sum()

Track               0
Artist              0
Duration            0
Time_Signature      0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
Popularity          0
Year                0
dtype: int64

mmmmmmmmmmmmmmmmmmmmmmmmm

In [10]:
covers_dataset.describe()

,Time_Signature,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Popularity,Year
count,919.000000,919.000000,919.000000,919.000000,919.000000,919.000000,919.000000,919.000000,919.000000,919.000000,919.000000,919.000000,919.000000,919.000000
mean,3.947769,0.622967,0.623548,5.412405,-8.349071,0.655060,0.075793,0.230068,0.052947,0.171865,0.531424,118.711789,57.722524,1994.405876
std,0.286769,0.154782,0.203975,3.514783,3.892821,0.475608,0.079570,0.256284,0.180214,0.138296,0.246091,29.684070,17.599522,2.926405
min,1.000000,0.135000,0.001340,0.000000,-42.950000,0.000000,0.023000,0.000004,0.000000,0.025000,0.006720,48.718000,0.000000,1990.000000
25%,4.000000,0.518500,0.479500,2.000000,-10.166500,0.000000,0.032400,0.029700,0.000000,0.089200,0.341500,96.924000,49.000000,1992.000000
50%,4.000000,0.631000,0.639000,6.000000,-7.599000,1.000000,0.042900,0.120000,0.000006,0.118000,0.535000,115.010000,60.000000,1994.000000
75%,4.000000,0.738000,0.780000,9.000000,-5.751000,1.000000,0.075600,0.344500,0.000741,0.217500,0.728500,133.424000,70.000000,1997.000000
max,5.000000,0.979000,0.999000,11.000000,0.919000,1.000000,0.529000,0.995000,0.987000,0.965000,0.975000,219.973000,92.000000,1999.000000


In [ ]:
covers_dataset.duplicated()